In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil
import cv2

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None


class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 14
controller.set_desired(set_speed)

cropped_height = 66
cropped_width = 320

# Format image to [66, 320, 1]
def format_image(X):
    X_proc = np.zeros((cropped_height, cropped_width, 1))
    
    for x in range(0, cropped_width-1):
        for y in range(0, cropped_height-1):
            X_proc[y, x, 0] = X[y, x]
    return X_proc


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
        image = image[70:70 + cropped_height, 0:cropped_width]
        image = format_image(image)
        X_train_data = np.zeros((1, cropped_height, cropped_width, 1))
        # image_array = np.asarray(image)
        # steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))
        X_train_data[0] =  format_image(image)

        steering_angle = float(model.predict(X_train_data, batch_size=1))

        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        # if args.image_folder != '':
            # timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            # image_filename = os.path.join(args.image_folder, timestamp)
            # image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':

    # check that model Keras version is same as local Keras version
    model_file = 'model.h5'
    # f = h5py.File(args.model, mode='r')
    f = h5py.File(model_file, mode='r')    
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    model = load_model(model_file)
    recored_images_folder = 'recorded_images'
    if recored_images_folder != '':
        print("Creating image folder at {}".format(recored_images_folder))
        if not os.path.exists(recored_images_folder):
            os.makedirs(recored_images_folder)
        else:
            shutil.rmtree(recored_images_folder)
            os.makedirs(recored_images_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's mi
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


Creating image folder at recorded_images
RECORDING THIS RUN ...
connect  62a9a95154814e728a7142f6cbadeb75
-0.1856565773487091 1.4280000000000002


(2331) wsgi starting up on http://0.0.0.0:4567
(2331) accepted ('127.0.0.1', 37641)


-0.1856565773487091 1.4560000000000002
-0.18239562213420868 1.4840000000000002
-0.18562889099121094 1.4924466
-0.19405998289585114 1.4732248000000001
-0.19405998289585114 1.4999230000000001
-0.24413013458251953 1.1978446
-0.21238334476947784 1.1704316000000001
-0.21238334476947784 1.1897486000000002
-0.22563092410564423 0.9040448
-0.17823553085327148 0.8480924
-0.17823553085327148 0.8600699999999999
-0.19478151202201843 0.5990344000000001
-0.19478151202201843 0.6056588
-0.19478151202201843 0.6122832
-0.08222556114196777 0.3704049999999999
-0.08222556114196777 0.37215679999999995
-0.08222556114196777 0.3739085999999999
0.09564170241355896 0.2030458
0.09564170241355896 0.201413
0.09564170241355896 0.19978020000000002
0.05360429361462593 0.11433400000000002
0.05360429361462593 0.11105780000000001
0.05360429361462593 0.1077816
0.1984788179397583 0.04368279999999991
0.1984788179397583 0.039213999999999916
0.1984788179397583 0.03474519999999992
0.22149063646793365 0.09724960000000002
0.22149

-0.1155247762799263 0.1496676000000001
-0.12509074807167053 0.15103820000000007
-0.12509074807167053 0.15154880000000004
-0.12843425571918488 0.09095120000000004
-0.13765954971313477 0.08742799999999994
-0.13765954971313477 0.08668479999999994
-0.21246573328971863 0.07541520000000002
-0.2107991874217987 0.1341662
-0.2107991874217987 0.13438720000000004
-0.2553083002567291 0.14489999999999997
-0.23275978863239288 0.14643460000000008
-0.23275978863239288 0.1468792000000001
-0.17529411613941193 0.14276440000000004
-0.17529411613941193 0.08781520000000011
-0.17529411613941193 0.08708600000000012
-0.06767481565475464 0.13148160000000003
-0.06832971423864365 0.13454419999999995
-0.06832971423864365 0.13475679999999995
-0.01819688454270363 0.14333340000000003
-0.058881815522909164 0.08910940000000007
-0.058881815522909164 0.08841540000000007
0.020602360367774963 0.08316200000000001
0.04126301035284996 0.13252180000000002
0.04126301035284996 0.1327216
0.0891186073422432 0.14664040000000012
0.0

-0.09740012139081955 0.17587580000000008
-0.09740012139081955 0.17706120000000009
-0.09740012139081955 0.1782466000000001
-0.20639382302761078 0.13045160000000008
-0.20639382302761078 0.1306766000000001
-0.20639382302761078 0.1309016000000001
-0.14026936888694763 0.15943160000000006
-0.14026936888694763 0.16021160000000007
-0.14026936888694763 0.16099160000000007
-0.2160172462463379 0.10827260000000008
-0.2160172462463379 0.10800360000000007
-0.2160172462463379 0.10773460000000007
-0.0036861319094896317 0.08569880000000007
-0.0036861319094896317 0.08500300000000008
-0.0036861319094896317 0.08430720000000008
-0.007151402533054352 0.14908520000000006
-0.007151402533054352 0.14967320000000006
-0.007151402533054352 0.15026120000000004
0.029452206566929817 0.1644764
-0.01192256435751915 0.11095540000000007
-0.01192256435751915 0.11074440000000008
-0.049867499619722366 0.151619
-0.03590674698352814 0.15293779999999993
-0.03590674698352814 0.15354659999999992
-0.08828332275152206 0.1018804000

-0.09427078813314438 0.09412180000000006
-0.07242954522371292 0.1438426000000001
-0.07242954522371292 0.1442734000000001
-0.17561432719230652 0.09970180000000009
-0.16803641617298126 0.15497279999999997
-0.16803641617298126 0.15561379999999997
-0.19010969996452332 0.15108340000000006
-0.2299700230360031 0.15438720000000014
-0.2299700230360031 0.15498100000000015
-0.2697538733482361 0.08733680000000009
-0.2697538733482361 0.13841879999999995
-0.2697538733482361 0.13869079999999995
-0.16967234015464783 0.07317280000000001
-0.16960947215557098 0.13024379999999997
-0.16960947215557098 0.13036479999999995
-0.20242533087730408 0.12451880000000008
-0.20242533087730408 0.07004460000000005
-0.20242533087730408 0.06898040000000005
-0.060740821063518524 0.06401980000000007
-0.1385483741760254 0.06347079999999997
-0.1385483741760254 0.06234179999999996
0.030536504462361336 0.05795900000000011
0.06951189041137695 0.09555679999999994
0.06951189041137695 0.09512459999999995
0.027751021087169647 0.112

-0.11018739640712738 0.1691722
-0.11018739640712738 0.1701266
-0.11018739640712738 0.17108099999999998
-0.0688125267624855 0.09872799999999995
-0.0688125267624855 0.09824499999999996
-0.0688125267624855 0.09776199999999995
0.1037469208240509 0.07656280000000003
0.1037469208240509 0.07567360000000004
0.1037469208240509 0.07478440000000003
0.1654096394777298 0.08446240000000005
0.1654096394777298 0.08378040000000003
0.1654096394777298 0.08309840000000004
0.27677443623542786 0.15448960000000003
0.16967399418354034 0.10220120000000002
0.16967399418354034 0.1018928
0.2935124337673187 0.16874120000000004
0.24118414521217346 0.1677504
0.24118414521217346 0.1687196
0.20303958654403687 0.1768797999999999
0.15997172892093658 0.11965619999999995
0.15997172892093658 0.11962259999999994
0.19925537705421448 0.10182059999999984
0.10841181874275208 0.10290739999999987
0.10841181874275208 0.10255419999999987
0.10039065778255463 0.15709739999999983
-0.009548529982566833 0.15887119999999993
-0.0095485299